In [27]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Add the correct relative path to the scripts folder
sys.path.append(os.path.abspath('../scripts'))
sys.path.append(os.path.abspath('../src'))

In [14]:
df = pd.read_csv('aggregated_data.csv')

In [15]:
df

,MSISDN/Number,Bearer Id,Dur. (ms),Total_Traffic
0,3.360100e+10,1.311448e+19,1.167200e+05,8.786906e+08
1,3.360100e+10,7.349883e+18,1.812300e+05,1.568596e+08
2,3.360100e+10,7.349883e+18,1.349690e+05,5.959665e+08
3,3.360101e+10,1.304243e+19,4.987800e+04,4.223207e+08
4,3.360101e+10,1.462771e+19,3.710400e+04,1.457411e+09
...,...,...,...,...
106850,3.378998e+10,2.615691e+19,2.103890e+05,1.094693e+09
106851,3.379000e+10,1.304243e+19,8.810000e+03,7.146416e+08
106852,3.379000e+10,1.311448e+19,1.409880e+05,4.803073e+08
106853,4.188282e+10,9.637368e+21,7.049061e+07,5.317447e+11


In [21]:
df_top =df.head(10)
df_top

,MSISDN/Number,Bearer Id,Dur. (ms),Total_Traffic
0,3.360100e+10,1.311448e+19,116720.0,8.786906e+08
1,3.360100e+10,7.349883e+18,181230.0,1.568596e+08
2,3.360100e+10,7.349883e+18,134969.0,5.959665e+08
3,3.360101e+10,1.304243e+19,49878.0,4.223207e+08
4,3.360101e+10,1.462771e+19,37104.0,1.457411e+09
5,3.360101e+10,1.462771e+19,253983.0,6.152172e+08
6,3.360101e+10,1.462771e+19,128360.0,6.547231e+08
7,3.360101e+10,1.304243e+19,86399.0,3.326604e+08
8,3.360101e+10,1.469977e+19,495702.0,9.901322e+08
9,3.360102e+10,7.349883e+18,124854.0,7.324638e+08


In [23]:
df_least =df.tail(10)
df_least

,MSISDN/Number,Bearer Id,Dur. (ms),Total_Traffic
106845,3.378994e+10,1.304243e+19,3.681200e+04,1.939172e+08
106846,3.378995e+10,2.622897e+19,2.090510e+05,1.540055e+09
106847,3.378996e+10,1.311448e+19,9.882600e+04,6.457777e+08
106848,3.378996e+10,1.311448e+19,1.570690e+05,7.102876e+08
106849,3.378997e+10,1.311448e+19,1.604610e+05,2.081231e+08
106850,3.378998e+10,2.615691e+19,2.103890e+05,1.094693e+09
106851,3.379000e+10,1.304243e+19,8.810000e+03,7.146416e+08
106852,3.379000e+10,1.311448e+19,1.409880e+05,4.803073e+08
106853,4.188282e+10,9.637368e+21,7.049061e+07,5.317447e+11
106854,3.197021e+12,7.277826e+18,2.415380e+05,2.321240e+08


In [16]:
data = pd.read_csv('aggregated_data_for_experiance.csv')

In [24]:
data_top= data.head(10)
data_top

,MSISDN/Number,TCP_Retransmission,RTT,Throughput
0,3.360100e+10,1137460.0,60.331977,76.0
1,3.360100e+10,1137460.0,31.000000,99.0
2,3.360100e+10,1137460.0,127.458589,97.0
3,3.360101e+10,2132.0,84.000000,248.0
4,3.360101e+10,9918360.0,66.665988,25496.0
5,3.360101e+10,571490.0,75.729294,3954.0
6,3.360101e+10,667008.0,33.665988,21256.5
7,3.360101e+10,556.0,52.000000,1247.0
8,3.360101e+10,1137460.0,127.458589,94.5
9,3.360102e+10,1137460.0,62.000000,146.0


In [38]:
data_least = data.tail(10)
data_least

,MSISDN/Number,TCP_Retransmission,RTT,Throughput
106845,3.378994e+10,5.428086e+06,52.000000,73497.000000
106846,3.378995e+10,1.137460e+06,45.500000,94.500000
106847,3.378996e+10,1.137460e+06,57.000000,89.000000
106848,3.378996e+10,1.137460e+06,127.458589,106.000000
106849,3.378997e+10,1.137460e+06,53.000000,67.000000
106850,3.378998e+10,5.688680e+05,98.229294,33937.500000
106851,3.379000e+10,4.300880e+05,52.000000,10365.000000
106852,3.379000e+10,1.137460e+06,40.000000,116.000000
106853,4.188282e+10,6.222656e+06,152.671824,16253.501588
106854,3.197021e+12,1.137460e+06,127.458589,1411.952463


In [47]:
user_metrics = np.array([ df]) 

In [48]:
least_engaged_centroid = np.array([data])

In [49]:
engagement_scores = np.linalg.norm(user_metrics - least_engaged_centroid, axis=1)

In [51]:
engagement_scores

array([[0.00000000e+00, 1.12360204e+22, 9.51892284e+07, 5.99614903e+11]])